In [0]:
# Environment: Google Colab with GPU

In [3]:
# Using Google Drive for storage.
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
cd /content/gdrive/My Drive/Hackathon/13Apr19

/content/gdrive/My Drive/Hackathon/13Apr19


In [0]:
import numpy as np
import pandas as pd




df_train           = pd.read_csv('train.csv')
df_test            = pd.read_csv('test.csv')

from sklearn.utils import shuffle
df_train = shuffle(df_train)








'''
df_train__loan_default__1     =     df_train[df_train['loan_default'] == 1]

extra = round(    ((len(df_train[df_train['loan_default'] == 0])/len(df_train[df_train['loan_default'] == 1])) % 1) * len(df_train[df_train['loan_default'] == 1])      )


df_train = df_train.append(df_train__loan_default__1, ignore_index=1)
df_train = df_train.append(df_train__loan_default__1, ignore_index=1)
df_train = df_train.append(df_train__loan_default__1[:extra], ignore_index=1)

df_train = shuffle(df_train)
'''










# len(df_train[df_train['loan_default'] == 1]), len(df_train[df_train['loan_default'] == 0])
# Data is not so imbalanced.

df_test['loan_default']  =  [0] * len(df_test)
df = df_train.append(df_test, ignore_index=1)











# for now - correlation study.
# df = df_train.copy()










'''   Formatting Columns - 'AVERAGE.ACCT.AGE' and 'CREDIT.HISTORY.LENGTH'   '''
df['AVERAGE.ACCT.AGE']         =    list([int(i.split('yrs ')[0]) * 12  +  int(i.split('yrs ')[1][:-3])    for i in df['AVERAGE.ACCT.AGE']])
df['CREDIT.HISTORY.LENGTH']    =    list([int(i.split('yrs ')[0]) * 12  +  int(i.split('yrs ')[1][:-3])    for i in df['CREDIT.HISTORY.LENGTH']])






'''   Creating features wtih column - 'Employment.Type'.   '''
df['Employment.Type'] = [str(i)  for i in df['Employment.Type']]


Employment_Type__LE__Dict = {'Salaried':0, 'Self employed':10}    # Weights.


# Average
Employment_Type__LE__Dict['nan'] = 5
df['Employment.Type__Average']  =  [Employment_Type__LE__Dict[i]  for i in df['Employment.Type']]


# Weighted Average
len_nan, len_Salaried, len_SE  =  len(df[df['Employment.Type'] == 'nan']), len(df[df['Employment.Type'] == 'Salaried']), len(df[df['Employment.Type'] == 'Self employed'])
Employment_Type__LE__Dict['nan'] = Employment_Type__LE__Dict['Salaried'] * (len_Salaried/(len_Salaried + len_SE))  + Employment_Type__LE__Dict['Self employed'] * (len_SE/(len_Salaried + len_SE))
df['Employment.Type__Weighted_Average'] = [Employment_Type__LE__Dict[i]  for i in df['Employment.Type']]


# Categorical
Employment_Type__LE__Dict = {'Salaried':0, 'nan':1, 'Self employed':2}
df['Employment.Type__Categorical'] = [Employment_Type__LE__Dict[i]  for i in df['Employment.Type']]






'''   Creating features wtih column - 'PERFORM_CNS.SCORE.DESCRIPTION'.   '''
count__PERFORM_CNS_SCORE_DESCRIPTION__Dict = {}    
for i in sorted(df['PERFORM_CNS.SCORE.DESCRIPTION'].unique()):
    count__PERFORM_CNS_SCORE_DESCRIPTION__Dict[i]  =  len(df[df['PERFORM_CNS.SCORE.DESCRIPTION'] == i])

# TODO: Experiment with these values to get even better correlation. [may research in thw web.]
# Assigning the values based on my intuition. 
PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict  =  {'A-Very Low Risk':0.05, 
                                             'B-Very Low Risk':0.1, 
                                             'C-Very Low Risk':0.3, 
                                             'D-Very Low Risk':0.8, 
                                             
                                             'E-Low Risk':1.1, 
                                             'F-Low Risk':1.4, 
                                             'G-Low Risk':1.8, 
                                             
                                             'H-Medium Risk':2.7,
                                             'I-Medium Risk':3.9,
                                             
                                             'J-High Risk':5,
                                             'K-High Risk':6,
                                             
                                             'L-Very High Risk':7.5,
                                             'M-Very High Risk':10}    # Weights.

count_of_AtoM  =  sum([                                                                                         count__PERFORM_CNS_SCORE_DESCRIPTION__Dict[i]                 for i in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict])
count__PERFORM_CNS_SCORE_DESCRIPTION__Dict____minus____PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict  =  [i  for i in count__PERFORM_CNS_SCORE_DESCRIPTION__Dict    if i not in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict]


# Weighted Average
weighted_avg   =  sum([PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i] * (count__PERFORM_CNS_SCORE_DESCRIPTION__Dict[i]/count_of_AtoM)  for i in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict])
for i in count__PERFORM_CNS_SCORE_DESCRIPTION__Dict____minus____PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict:
        PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  =  weighted_avg

df['PERFORM_CNS.SCORE.DESCRIPTION__Weighted_Average'] = [PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  for i in df['PERFORM_CNS.SCORE.DESCRIPTION']]


# Average
_avg   =  sum([PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]   for i in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict]) / len(PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict)
for i in count__PERFORM_CNS_SCORE_DESCRIPTION__Dict____minus____PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict:
        PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  =  _avg

df['PERFORM_CNS.SCORE.DESCRIPTION__Average'] = [PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  for i in df['PERFORM_CNS.SCORE.DESCRIPTION']]


# Categorical
PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict  =  dict((value, key) for (key, value) in enumerate(sorted(df['PERFORM_CNS.SCORE.DESCRIPTION'].unique())))
df['PERFORM_CNS.SCORE.DESCRIPTION__Categorical'] = [PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  for i in df['PERFORM_CNS.SCORE.DESCRIPTION']]

# Bucketed Categorical
for i in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict:    PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i] = min(PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i], 14)
df['PERFORM_CNS.SCORE.DESCRIPTION__Bucketed_Categorical__14'] = [PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  for i in df['PERFORM_CNS.SCORE.DESCRIPTION']]

for i in PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict:    PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i] = min(PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i], 13)
df['PERFORM_CNS.SCORE.DESCRIPTION__Bucketed_Categorical__13'] = [PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict[i]  for i in df['PERFORM_CNS.SCORE.DESCRIPTION']]








'''   Creating features with column -  'Date.of.Birth' and 'DisbursalDate'.   '''

def format_date(col):
    temp = []
    for x in pd.to_datetime(col, format="%d-%m-%y"):
        if x > pd.to_datetime('today'):    temp.append(x - pd.DateOffset(years=100))
        else:                              temp.append(x)
    return temp


df['Date.of.Birth']  =  format_date(df['Date.of.Birth'])
df['DOB_day']    =  df['Date.of.Birth'].dt.day
df['DOB_month']  =  df['Date.of.Birth'].dt.month
df['DOB_year']   =  df['Date.of.Birth'].dt.year
df['DOB_dayofweek']   =  df['Date.of.Birth'].dt.dayofweek

df['DisbursalDate']  =  format_date(df['DisbursalDate'])
df['DisbursalDate_day']    =  df['DisbursalDate'].dt.day
df['DisbursalDate_month']  =  df['DisbursalDate'].dt.month
df['DisbursalDate_year']   =  df['DisbursalDate'].dt.year
df['DisbursalDate_dayofweek']   =  df['DisbursalDate'].dt.dayofweek

df['Age__at_loan_disbursal']  =  df['DisbursalDate'] - df['Date.of.Birth']
df['number_of_days__at_loan_disbursal']  =  [int(str(i).replace(' days 00:00:00', ''))  for i in df['Age__at_loan_disbursal']]






''' Too many people are born on 1st and  in Jan.

[(i, round(list(df['DOB_day']).count(i)/len(df), 2)*100  )  for i in set(list(df['DOB_day']))]

[(i, round(list(df['DOB_month']).count(i)/len(df), 2)*100  )  for i in set(list(df['DOB_month']))]

'''






'''   Creating features with column -  'PRIMARY.INSTAL.AMT' and 'SEC.INSTAL.AMT'.   '''
df['PRIMARY.INSTAL.AMT__minus__SEC.INSTAL.AMT']    =  df['PRIMARY.INSTAL.AMT'] - df['SEC.INSTAL.AMT']
df['PRIMARY.INSTAL.AMT__by__SEC.INSTAL.AMT']       =  df['PRIMARY.INSTAL.AMT'] / (df['SEC.INSTAL.AMT'] + 100000)    # A random value - can experiment with 10**k, k in [0, 10].


'''   CREATING MANY COLUMNS   '''

l = ['PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS']
for i in range(len(l)-1):
    for j in range(i+1, len(l)):
        df[l[i] + '__minus__' + l[j]]  =  df[l[i]] - df[l[j]]
        df[l[i] + '__by__' + l[j]]  =  df[l[i]] / (df[l[j]] + 1)
        
        
l = ['PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT']
for i in range(len(l)-1):
    for j in range(i+1, len(l)):
        df[l[i] + '__minus__' + l[j]]  =  df[l[i]] - df[l[j]]
        df[l[i] + '__by__' + l[j]]  =  df[l[i]] / (df[l[j]] + 100000)
        
      
    
#corr = df.corr()


columns_with_same_values = ['MobileNo_Avl_Flag', 'DisbursalDate_year']    # saw in the correlation

for i in columns_with_same_values: del df[i]




# method : {‘pearson’, ‘kendall’, ‘spearman’} 
#corr_pearson   =  df.corr(method = 'pearson')
#corr_kendall   =  df.corr(method = 'kendall')
#corr_spearman  =  df.corr(method = 'spearman')





In [0]:




'''   Arranging Columns   '''
#list(df)



arranged_columns =  ['UniqueID',
                     
                     'disbursed_amount',
                     'asset_cost',
                     'ltv',
                     
                     'branch_id',
                     'supplier_id',
                     'manufacturer_id',
                     'Current_pincode_ID',


                     'Employment.Type',
                     'Employment.Type__Average',
                     'Employment.Type__Weighted_Average',
                     'Employment.Type__Categorical',
                    
                    
                     'Date.of.Birth',
                     'DOB_day',
                     'DOB_month',
                     'DOB_year',
                     'DOB_dayofweek',


                     'DisbursalDate',
                     'DisbursalDate_day',
                     'DisbursalDate_month',
                     'DisbursalDate_dayofweek',
                    
                                         
                     'Age__at_loan_disbursal',
                     'number_of_days__at_loan_disbursal',
                    
                     'State_ID',
                     'Employee_code_ID',
                    
                     'Aadhar_flag',
                     'PAN_flag',
                     'VoterID_flag',
                     'Driving_flag',
                     'Passport_flag',
                    
                     'PERFORM_CNS.SCORE',
                     'PERFORM_CNS.SCORE.DESCRIPTION',
                    
                    
                    
                     'PRI.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS',
                     'PRI.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT',
                     'SEC.NO.OF.ACCTS',
                     'SEC.ACTIVE.ACCTS',
                     'SEC.OVERDUE.ACCTS',
                     'SEC.CURRENT.BALANCE',
                     'SEC.SANCTIONED.AMOUNT',
                     'SEC.DISBURSED.AMOUNT',
                     'PRI.NO.OF.ACCTS__minus__PRI.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__PRI.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__PRI.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__PRI.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__PRI.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__PRI.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'SEC.NO.OF.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'SEC.NO.OF.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'SEC.NO.OF.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'SEC.NO.OF.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'SEC.ACTIVE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'SEC.ACTIVE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.CURRENT.BALANCE__minus__PRI.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__PRI.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__PRI.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__PRI.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__SEC.CURRENT.BALANCE',
                     'PRI.CURRENT.BALANCE__by__SEC.CURRENT.BALANCE',
                     'PRI.CURRENT.BALANCE__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__SEC.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__PRI.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__PRI.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.CURRENT.BALANCE',
                     'PRI.DISBURSED.AMOUNT__by__SEC.CURRENT.BALANCE',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                     'SEC.CURRENT.BALANCE__minus__SEC.SANCTIONED.AMOUNT',
                     'SEC.CURRENT.BALANCE__by__SEC.SANCTIONED.AMOUNT',
                     'SEC.CURRENT.BALANCE__minus__SEC.DISBURSED.AMOUNT',
                     'SEC.CURRENT.BALANCE__by__SEC.DISBURSED.AMOUNT',
                     'SEC.SANCTIONED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'SEC.SANCTIONED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                    
                    
                    
                     'PRIMARY.INSTAL.AMT',
                     'SEC.INSTAL.AMT',
                     'PRIMARY.INSTAL.AMT__minus__SEC.INSTAL.AMT',
                     'PRIMARY.INSTAL.AMT__by__SEC.INSTAL.AMT',
                     
                     
                     
                     'NEW.ACCTS.IN.LAST.SIX.MONTHS',
                     'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
                     'AVERAGE.ACCT.AGE',
                     'CREDIT.HISTORY.LENGTH',
                     'NO.OF_INQUIRIES',
                    
                     
                     
                     'PERFORM_CNS.SCORE.DESCRIPTION__Weighted_Average',
                     'PERFORM_CNS.SCORE.DESCRIPTION__Average',
                     'PERFORM_CNS.SCORE.DESCRIPTION__Categorical',
                     'PERFORM_CNS.SCORE.DESCRIPTION__Bucketed_Categorical__14',
                     'PERFORM_CNS.SCORE.DESCRIPTION__Bucketed_Categorical__13',
                    
                    
                     'loan_default']


df = df[arranged_columns]
#corr = df.corr()



subset_columns   =  ['disbursed_amount',
                     'asset_cost',
                     'ltv',
                     
                     'branch_id',
                     'supplier_id',
                     'manufacturer_id',
                     'Current_pincode_ID',


                     'Employment.Type__Categorical',
                    
                    
                     'DOB_day',
                     'DOB_month',
                     'DOB_year',
                     'DOB_dayofweek',


                     'DisbursalDate_day',
                     'DisbursalDate_month',
                     'DisbursalDate_dayofweek',
                    
                     'number_of_days__at_loan_disbursal',
                    
                     'State_ID',
                     'Employee_code_ID',
                    
                     'Aadhar_flag',
                     'PAN_flag',
                     'VoterID_flag',
                     'Driving_flag',
                     'Passport_flag',
                    
                     'PERFORM_CNS.SCORE',

                     'PERFORM_CNS.SCORE.DESCRIPTION__Weighted_Average',
                         
                    
                    
                     'PRI.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS',
                     'PRI.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT',
                     'SEC.NO.OF.ACCTS',
                     'SEC.ACTIVE.ACCTS',
                     'SEC.OVERDUE.ACCTS',
                     'SEC.CURRENT.BALANCE',
                     'SEC.SANCTIONED.AMOUNT',
                     'SEC.DISBURSED.AMOUNT',
                     'PRI.NO.OF.ACCTS__minus__PRI.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__PRI.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__PRI.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__PRI.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.NO.OF.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.NO.OF.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__PRI.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__PRI.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.ACTIVE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.ACTIVE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.NO.OF.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.NO.OF.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'PRI.OVERDUE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'PRI.OVERDUE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'SEC.NO.OF.ACCTS__minus__SEC.ACTIVE.ACCTS',
                     'SEC.NO.OF.ACCTS__by__SEC.ACTIVE.ACCTS',
                     'SEC.NO.OF.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'SEC.NO.OF.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'SEC.ACTIVE.ACCTS__minus__SEC.OVERDUE.ACCTS',
                     'SEC.ACTIVE.ACCTS__by__SEC.OVERDUE.ACCTS',
                     'PRI.CURRENT.BALANCE__minus__PRI.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__PRI.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__PRI.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__PRI.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__SEC.CURRENT.BALANCE',
                     'PRI.CURRENT.BALANCE__by__SEC.CURRENT.BALANCE',
                     'PRI.CURRENT.BALANCE__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.CURRENT.BALANCE__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.CURRENT.BALANCE__by__SEC.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__PRI.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__PRI.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.SANCTIONED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.CURRENT.BALANCE',
                     'PRI.DISBURSED.AMOUNT__by__SEC.CURRENT.BALANCE',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__by__SEC.SANCTIONED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'PRI.DISBURSED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                     'SEC.CURRENT.BALANCE__minus__SEC.SANCTIONED.AMOUNT',
                     'SEC.CURRENT.BALANCE__by__SEC.SANCTIONED.AMOUNT',
                     'SEC.CURRENT.BALANCE__minus__SEC.DISBURSED.AMOUNT',
                     'SEC.CURRENT.BALANCE__by__SEC.DISBURSED.AMOUNT',
                     'SEC.SANCTIONED.AMOUNT__minus__SEC.DISBURSED.AMOUNT',
                     'SEC.SANCTIONED.AMOUNT__by__SEC.DISBURSED.AMOUNT',
                    
                    
                    
                     'PRIMARY.INSTAL.AMT',
                     'SEC.INSTAL.AMT',
                     'PRIMARY.INSTAL.AMT__minus__SEC.INSTAL.AMT',
                     'PRIMARY.INSTAL.AMT__by__SEC.INSTAL.AMT',
                     
                     
                     
                     'NEW.ACCTS.IN.LAST.SIX.MONTHS',
                     'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
                     'AVERAGE.ACCT.AGE',
                     'CREDIT.HISTORY.LENGTH',
                     'NO.OF_INQUIRIES',
                    
                     
                    
                     'loan_default']



df_subset = df[subset_columns]
#corr_subset = df_subset.corr()



In [0]:


X = df_subset.iloc[:, 0:len(list(df_subset)) - 1].values
y = df_subset.iloc[:,   len(list(df_subset)) - 1].values


#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)
X_train, X_test  =  X[:len(df_train)], X[len(df_train):]
y_train, y_test  =  y[:len(df_train)], y[len(df_train):]



In [0]:
#print(len(df_train), len(df_test), len(df),
#len(X_train), len(X_test), len(X),
#len(y_train), len(y_test), len(y))



In [0]:
del Employment_Type__LE__Dict, PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict,	X, arranged_columns, columns_with_same_values, count__PERFORM_CNS_SCORE_DESCRIPTION__Dict, count__PERFORM_CNS_SCORE_DESCRIPTION__Dict____minus____PERFORM_CNS_SCORE_DESCRIPTION__LE__Dict, count_of_AtoM, df, df_subset, df_test, df_train, l, len_SE, len_Salaried, len_nan, subset_columns, weighted_avg, y_test

In [9]:
!pip install catboost -U

    100% |████████████████████████████████| 60.6MB 804kB/s 


In [10]:

from catboost import CatBoostRegressor, CatBoostClassifier
classifier = CatBoostClassifier(iterations = 10000,
                                task_type = 'GPU',
                                eval_metric='AUC',
                                loss_function= 'Logloss',
                                cat_features=[3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22],
                                #learning_rate=0.003
                                scale_pos_weight = 1.55,
                                #depth=7
                                )

classifier.fit(X_train, y_train)

Learning rate set to 0.010073
0:	learn: 0.6244170	total: 77.7ms	remaining: 12m 56s
1:	learn: 0.6278301	total: 152ms	remaining: 12m 38s
2:	learn: 0.6312402	total: 212ms	remaining: 11m 45s
3:	learn: 0.6351700	total: 274ms	remaining: 11m 24s
4:	learn: 0.6360702	total: 342ms	remaining: 11m 23s
5:	learn: 0.6374307	total: 392ms	remaining: 10m 52s
6:	learn: 0.6382750	total: 442ms	remaining: 10m 30s
7:	learn: 0.6391727	total: 496ms	remaining: 10m 18s
8:	learn: 0.6404064	total: 552ms	remaining: 10m 12s
9:	learn: 0.6404013	total: 604ms	remaining: 10m 3s
10:	learn: 0.6409905	total: 654ms	remaining: 9m 53s
11:	learn: 0.6412396	total: 703ms	remaining: 9m 44s
12:	learn: 0.6406480	total: 758ms	remaining: 9m 42s
13:	learn: 0.6407205	total: 816ms	remaining: 9m 42s
14:	learn: 0.6413370	total: 857ms	remaining: 9m 30s
15:	learn: 0.6414764	total: 902ms	remaining: 9m 22s
16:	learn: 0.6413630	total: 952ms	remaining: 9m 18s
17:	learn: 0.6416197	total: 1s	remaining: 9m 15s
18:	learn: 0.6418718	total: 1.05s	rem

In [0]:
import time
time.sleep(20)

y_pred = classifier.predict_proba(X_test)
y_pred = [i[1]  for i in y_pred]


#from sklearn.metrics import roc_auc_score
#print(roc_auc_score(y_test, y_pred))
df__sample_submission           = pd.read_csv('sample_submission.csv')
df__sample_submission['loan_default']  =  y_pred
df__sample_submission.to_csv('submission10.csv', index=0)
